# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'10-25-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'10-25-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-10-26 04:25:02,33.93911,67.709953,40833,1514,34129,5190.0,Afghanistan,104.892731,3.707785
1,NaN,NaN,NaN,Albania,2020-10-26 04:25:02,41.15330,20.168300,19157,477,10654,8026.0,Albania,665.682118,2.489951
2,NaN,NaN,NaN,Algeria,2020-10-26 04:25:02,28.03390,1.659600,56143,1914,39095,15134.0,Algeria,128.031162,3.409152
3,NaN,NaN,NaN,Andorra,2020-10-26 04:25:02,42.50630,1.521800,4038,69,2729,1240.0,Andorra,5226.169676,1.708767
4,NaN,NaN,NaN,Angola,2020-10-26 04:25:02,-11.20270,17.873900,9381,268,3508,5605.0,Angola,28.542943,2.856838


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,40073,40141,40200,40287,40357,40510,40626,40687,40768,40833
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,16501,16774,17055,17350,17651,17948,18250,18556,18858,19157
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,53998,54203,54402,54616,54829,55081,55357,55630,55880,56143


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1485,1488,1492,1497,1499,1501,1505,1507,1511,1514
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,443,448,451,454,458,462,465,469,473,477
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1841,1846,1856,1865,1873,1880,1888,1897,1907,1914


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,33516,33561,33614,33760,33790,33824,33831,34010,34023,34129
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,9957,10001,10071,10167,10225,10341,10395,10466,10548,10654
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,37856,37971,38088,38215,38346,38482,38618,38788,38932,39095


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1966,1983,1989,1999,2010,2021,2023,2030,2048,2059
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,6333,6350,6369,6375,6405,6443,6475,6615,6637,6658
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,968,977,981,981,988,996,997,1012,1031,1033


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,28,28,28,28,29,30,30,30,31,31
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,67,67,67,67,69,69,69,69,69,69
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,9,9,9,9,9,9,9,9,9,9


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
633,1001.0,Autauga,Alabama,US,2020-10-26 04:25:02,32.539527,-86.644082,2059,31,0,2028.0,"Autauga, Alabama, US",3685.406934,1.505585
670,1075.0,Lamar,Alabama,US,2020-10-26 04:25:02,33.779950,-88.096680,496,5,0,491.0,"Lamar, Alabama, US",3592.901123,1.008065
671,1077.0,Lauderdale,Alabama,US,2020-10-26 04:25:02,34.901719,-87.656247,2294,42,0,2252.0,"Lauderdale, Alabama, US",2473.875487,1.830863


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,8635966,225229,3422878
India,7909959,119014,7137228
Brazil,5380635,156903,4526393


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,8635966,225229,3422878,4987859,2020-10-26 04:25:02,37.936303,-91.379001
India,7909959,119014,7137228,653717,2020-10-26 04:25:02,23.088275,81.806127
Brazil,5380635,156903,4526393,697339,2020-10-26 04:25:02,-12.669522,-48.480493
Russia,1503652,25875,1132790,344987,2020-10-26 04:25:02,54.546312,62.120860
France,1130143,34673,114534,980936,2020-10-26 04:25:02,8.314863,-21.695174


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,906596,17358,0
Texas,892006,17955,0
Florida,778636,16429,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,906596,17358,0,889238,2020-10-26 04:25:02,37.843962,-120.728594
Texas,892006,17955,0,874051,2020-10-26 04:25:02,31.660643,-98.653069
Florida,778636,16429,0,762207,2020-10-26 04:25:02,28.940755,-82.700744
New York,495464,33422,0,462042,2020-10-26 04:25:02,42.544151,-75.474183
Illinois,378958,9775,0,369183,2020-10-26 04:25:02,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,299760,6993,0
Florida,Miami-Dade,181942,3612,0
Illinois,Cook,175586,5421,0
Texas,Harris,158758,2783,0
Arizona,Maricopa,154182,3540,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,299760,6993,0,292767,2020-10-26 04:25:02,34.308284,-118.228241,6037.0
Florida,Miami-Dade,181942,3612,0,178330,2020-10-26 04:25:02,25.611236,-80.551706,12086.0
Illinois,Cook,175586,5421,0,170165,2020-10-26 04:25:02,41.841448,-87.816588,17031.0
Texas,Harris,158758,2783,0,155975,2020-10-26 04:25:02,29.858649,-95.393395,48201.0
Arizona,Maricopa,154182,3540,0,150642,2020-10-26 04:25:02,33.348359,-112.491815,4013.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-23,40687,18556,55630,4038,8829,122,1069368,73310,27495,74415,...,834010,2759,64724,88718,1148,49579,10,2060,16095,8257
2020-10-24,40768,18858,55880,4038,9026,124,1081336,75523,27513,78029,...,857043,2807,64923,89142,1160,49989,10,2060,16117,8269
2020-10-25,40833,19157,56143,4038,9381,124,1090589,77837,27525,80811,...,876840,2851,65307,89565,1168,50442,10,2060,16117,8276


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-23,1507,469,1897,69,265,3,28338,1145,905,954,...,44661,53,542,764,35,439,1,599,346,236
2020-10-24,1511,473,1907,69,267,3,28613,1157,905,965,...,44835,53,544,770,35,443,1,599,348,236
2020-10-25,1514,477,1914,69,268,3,28896,1180,905,979,...,44986,53,549,773,35,448,1,599,348,237


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-23,34010,10466,38788,2729,3384,107,866695,50276,25173,55195,...,2657,2241,61957,82899,1049,42850,8,1354,15179,7771
2020-10-24,34023,10548,38932,2729,3461,108,881113,50701,25179,56791,...,2676,2301,62033,83443,1051,43232,8,1360,15179,7785
2020-10-25,34129,10654,39095,2729,3508,108,894819,50908,25196,57858,...,2685,2351,62509,83947,1057,43592,8,1360,15179,7797


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1966,1983,1989,1999,2010,2021,2023,2030,2048,2059
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,441,446,455,457,456,459,466,474,490,496
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,1994,2029,2043,2063,2087,2125,2148,2180,2262,2294
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,772,782,788,792,800,811,821,837,850,858
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,6400,6421,6430,6439,6449,6461,6496,6517,6534,6546


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-23,180916,12113,0,235882,104135,898799,91549,66052,23687,49,...,241513,878567,101509,2016,1346,170730,101345,21393,190478,10545
2020-10-24,183276,12485,0,236772,105318,903684,93377,66052,23847,49,...,244087,888362,103117,2043,1346,171791,102264,21716,194540,10805
2020-10-25,184355,13008,0,238163,106115,906596,95066,66052,23961,49,...,247587,892006,104882,2073,1346,172774,102913,21907,198166,11041


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-10-23        2030    6615    1012  825   1911     639   1002    4224   
2020-10-24        2048    6637    1031  828   1925     648   1007    4567   
2020-10-25        2059    6658    1033  840   1932     649   1011    4599   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-10-23         1343      731  ...     536    115      634      158   
2020-10-24         1350      733  ...     547    117      659      158   
2020-10-25         1350      738  ...     588    122      670      159   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-10-23            451   740   425          0      144    121  
2020-10-24            456   743   428          0      145    127  
2020-10-25            462   747   431          0      145    127  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-23,2859,68,0,5865,1782,17305,2211,4577,678,0,...,3076,17773,567,58,21,3535,2296,424,1745,68
2020-10-24,2866,68,0,5869,1797,17343,2218,4577,680,0,...,3100,17915,568,58,21,3574,2296,424,1770,68
2020-10-25,2866,68,0,5874,1812,17358,2223,4577,681,0,...,3131,17955,572,58,21,3575,2296,425,1778,68


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-10-23          30      69       9   14     25      17     40      61   
2020-10-24          31      69       9   14     25      17     40      61   
2020-10-25          31      69       9   14     25      17     40      61   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-10-23           43       14  ...       3      1        4        1   
2020-10-24           44       14  ...       3      1        4        1   
2020-10-25           44       14  ...       3      1        4        1   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-10-23              2     1     3          0        7      0  
2020-10-24              2     1     3          0        7      0  
2020-10-25              2     1     3          0        7      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-23,0.001502,0.016767,0.004932,0.059564,0.028781,0.000000,0.014918,0.034926,0.000692,0.035786,...,0.025274,0.021474,0.004423,0.003416,0.000000,0.009057,0.0,0.001458,0.003742,0.001820
2020-10-24,0.001991,0.016275,0.004494,0.000000,0.022313,0.016393,0.011192,0.030187,0.000655,0.048565,...,0.027617,0.017398,0.003075,0.004779,0.010453,0.008270,0.0,0.000000,0.001367,0.001453
2020-10-25,0.001594,0.015855,0.004707,0.000000,0.039331,0.000000,0.008557,0.030640,0.000436,0.035653,...,0.023099,0.015675,0.005915,0.004745,0.006897,0.009062,0.0,0.000000,0.000000,0.000847


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-23,0.001329,0.008602,0.004767,0.095238,0.019231,0.0,0.013628,0.012378,0.0,0.013815,...,0.005041,0.0,0.003704,0.006588,0.0,0.009195,0.0,0.00335,0.00000,0.000000
2020-10-24,0.002654,0.008529,0.005271,0.000000,0.007547,0.0,0.009704,0.010480,0.0,0.011530,...,0.003896,0.0,0.003690,0.007853,0.0,0.009112,0.0,0.00000,0.00578,0.000000
2020-10-25,0.001985,0.008457,0.003671,0.000000,0.003745,0.0,0.009891,0.019879,0.0,0.014508,...,0.003368,0.0,0.009191,0.003896,0.0,0.011287,0.0,0.00000,0.00000,0.004237


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-23,0.005291,0.006830,0.004402,0.104858,0.023903,0.000000,0.017422,0.009822,0.000477,0.022698,...,0.001885,0.016788,0.004849,0.007474,0.000000,0.007193,0.0,0.007440,0.000725,0.003746
2020-10-24,0.000382,0.007835,0.003712,0.000000,0.022754,0.009346,0.016636,0.008453,0.000238,0.028916,...,0.007151,0.026774,0.001227,0.006562,0.001907,0.008915,0.0,0.004431,0.000000,0.001802
2020-10-25,0.003116,0.010049,0.004187,0.000000,0.013580,0.000000,0.015555,0.004083,0.000675,0.018788,...,0.003363,0.021730,0.007673,0.006040,0.005709,0.008327,0.0,0.000000,0.000000,0.001541


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-23,0.021755,0.023490,NaN,0.004155,0.013006,0.006084,0.014967,0.010387,0.006758,0.0,...,0.015157,0.008163,0.019689,0.014595,0.002234,0.006865,0.008157,0.015957,0.023525,0.042099
2020-10-24,0.013045,0.030711,NaN,0.003773,0.011360,0.005435,0.019967,0.000000,0.006755,0.0,...,0.010658,0.011149,0.015841,0.013393,0.000000,0.006214,0.009068,0.015098,0.021325,0.024656
2020-10-25,0.005887,0.041890,NaN,0.005875,0.007568,0.003222,0.018088,0.000000,0.004780,0.0,...,0.014339,0.004102,0.017116,0.014684,0.000000,0.005722,0.006346,0.008795,0.018639,0.021842


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-10-23      0.003460  0.021622  0.015045  0.017263  0.009509  0.003140   
2020-10-24      0.008867  0.003326  0.018775  0.003636  0.007326  0.014085   
2020-10-25      0.005371  0.003164  0.001940  0.014493  0.003636  0.001543   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-10-23      0.000999  0.008355  0.005240  0.006887  ...  0.063492   
2020-10-24      0.004990  0.081203  0.005212  0.002736  ...  0.020522   
2020-10-25      0.003972  0.007007  0.000000  0.006821  ...  0.074954   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-10-23      0.017699  0.060201  0.006369   0.018059  0.017882  0.036585   
2020-10-24      0.017391  0.039432  0.000000   0.011086  0.004054  0.007059   
2020-10-25      0.042735  0.016692  0.006329   0.013158  0.005384  0.007009   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-10-23            NaN  0.006993  0.052174  
2020-10-24            NaN  0.006944  0.049587  
2020-10-25            NaN  0.000000  0.000000  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-23,0.005628,0.0,NaN,0.001024,0.005643,0.002840,0.005914,0.001751,0.011940,NaN,...,0.021588,0.006456,0.007105,0.0,0.0,0.004261,0.003058,0.009524,0.024662,0.0
2020-10-24,0.002448,0.0,NaN,0.000682,0.008418,0.002196,0.003166,0.000000,0.002950,NaN,...,0.007802,0.007990,0.001764,0.0,0.0,0.011033,0.000000,0.000000,0.014327,0.0
2020-10-25,0.000000,0.0,NaN,0.000852,0.008347,0.000865,0.002254,0.000000,0.001471,NaN,...,0.010000,0.002233,0.007042,0.0,0.0,0.000280,0.000000,0.002358,0.004520,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                       \
Admin2           Autauga Baldwin Barbour Bibb Blount Bullock Butler   Calhoun   
2020-10-23      0.000000     0.0     0.0  0.0    0.0     0.0    0.0  0.016667   
2020-10-24      0.033333     0.0     0.0  0.0    0.0     0.0    0.0  0.000000   
2020-10-25      0.000000     0.0     0.0  0.0    0.0     0.0    0.0  0.000000   

Province_State                     ... Wyoming                           \
Admin2          Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-10-23      0.000000      0.0  ...     0.0    0.0      0.0      0.0   
2020-10-24      0.023256      0.0  ...     0.0    0.0      0.0      0.0   
2020-10-25      0.000000      0.0  ...     0.0    0.0      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-10-23            0.0   0.0   0.5       -1.0      0.0    NaN  
2020-10-24            0.0   0.0   0.0        NaN      0.0    NaN  
2020-10-25            0.0   0.0   0.0        NaN      0.0    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-23,0.002162,0.016834,0.004776,0.038867,0.029584,0.003649,0.015480,0.031650,0.000684,0.033081,...,0.027072,0.018565,0.004884,0.004224,0.001561,0.009667,5.224419e-39,0.000771,0.003118,0.002522
2020-10-24,0.002077,0.016555,0.004635,0.019434,0.025948,0.010021,0.013336,0.030919,0.000669,0.040823,...,0.027345,0.017981,0.003979,0.004502,0.006007,0.008969,2.612210e-39,0.000386,0.002243,0.001988
2020-10-25,0.001835,0.016205,0.004671,0.009717,0.032640,0.005011,0.010947,0.030779,0.000553,0.038238,...,0.025222,0.016828,0.004947,0.004624,0.006452,0.009015,1.306105e-39,0.000193,0.001121,0.001417


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-23,0.001756,0.008047,0.004479,0.051268,0.018158,1.605538e-58,0.014491,0.011901,0.000020,0.013996,...,0.004606,0.003636,0.005532,0.007453,1.766155e-17,0.012957,0.0,0.001702,0.000012,0.002019
2020-10-24,0.002205,0.008288,0.004875,0.025634,0.012853,8.027690e-59,0.012098,0.011191,0.000010,0.012763,...,0.004251,0.001818,0.004611,0.007653,8.830776e-18,0.011034,0.0,0.000851,0.002896,0.001009
2020-10-25,0.002095,0.008372,0.004273,0.012817,0.008299,4.013845e-59,0.010994,0.015535,0.000005,0.013636,...,0.003809,0.000909,0.006901,0.005775,4.415388e-18,0.011161,0.0,0.000425,0.001448,0.002623


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-23,0.003067,0.007012,0.003951,0.066693,0.034290,0.014960,0.016228,0.009977,0.000533,0.023112,...,0.003688,0.015198,0.005899,0.008580,0.001181,0.010113,3.061183e-41,0.004328,0.00164,0.003117
2020-10-24,0.001725,0.007424,0.003832,0.033347,0.028522,0.012153,0.016432,0.009215,0.000386,0.026014,...,0.005419,0.020986,0.003563,0.007571,0.001544,0.009514,1.530592e-41,0.004380,0.00082,0.002459
2020-10-25,0.002420,0.008736,0.004009,0.016673,0.021051,0.006076,0.015994,0.006649,0.000531,0.022401,...,0.004391,0.021358,0.005618,0.006805,0.003626,0.008920,7.652958e-42,0.002190,0.00041,0.002000


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-23,0.014406,0.021447,NaN,0.004153,0.011854,0.005824,0.014813,0.009017,0.005993,1.010167e-67,...,0.012153,0.006461,0.017057,0.010932,0.002370,0.006774,0.007285,0.014661,0.022654,0.035325
2020-10-24,0.013726,0.026079,NaN,0.003963,0.011607,0.005629,0.017390,0.004509,0.006374,5.050834e-68,...,0.011406,0.008805,0.016449,0.012162,0.001185,0.006494,0.008176,0.014880,0.021990,0.029990
2020-10-25,0.009806,0.033985,NaN,0.004919,0.009587,0.004426,0.017739,0.002254,0.005577,2.525417e-68,...,0.012872,0.006453,0.016783,0.013423,0.000593,0.006108,0.007261,0.011837,0.020314,0.025916


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-10-23      0.003340  0.013227  0.009415  0.013996  0.009987  0.002073   
2020-10-24      0.006104  0.008276  0.014095  0.008816  0.008657  0.008079   
2020-10-25      0.005737  0.005720  0.008017  0.011654  0.006147  0.004811   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-10-23      0.001473  0.007711  0.008122  0.005526  ...  0.053243   
2020-10-24      0.003232  0.044457  0.006667  0.004131  ...  0.036883   
2020-10-25      0.003602  0.025732  0.003334  0.005476  ...  0.055919   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-10-23      0.046389  0.052598  0.014869   0.017037  0.014124  0.021618   
2020-10-24      0.031890  0.046015  0.007434   0.014062  0.009089  0.014338   
2020-10-25      0.037313  0.031354  0.006882   0.013610  0.007236  0.010674   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-10-23      -0.999999  0.010233  0.063333  
2020-10-24      -0.999999  0.008589  0.056460  
2020-10-25      -0.999999  0.004294  0.028230  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-23,0.005635,0.002140,NaN,0.001232,0.008540,0.003891,0.004262,0.00136,0.006640,NaN,...,0.015960,0.004393,0.008563,2.925468e-26,0.000098,0.004653,0.003037,0.010847,0.021190,0.033102
2020-10-24,0.004042,0.001070,NaN,0.000957,0.008479,0.003043,0.003714,0.00068,0.004795,NaN,...,0.011881,0.006191,0.005163,1.462734e-26,0.000049,0.007843,0.001518,0.005424,0.017758,0.016551
2020-10-25,0.002021,0.000535,NaN,0.000904,0.008413,0.001954,0.002984,0.00034,0.003133,NaN,...,0.010940,0.004212,0.006103,7.313670e-27,0.000024,0.004061,0.000759,0.003891,0.011139,0.008275


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-10-23      0.006543  0.001943  0.000004  0.004892  0.008353  0.000122   
2020-10-24      0.019938  0.000972  0.000002  0.002446  0.004177  0.000061   
2020-10-25      0.009969  0.000486  0.000001  0.001223  0.002088  0.000031   

Province_State                                                  ...   Wyoming  \
Admin2                Butler   Calhoun      Chambers  Cherokee  ...      Park   
2020-10-23      7.765147e-10  0.012803  1.816524e-07  0.000075  ...  0.031250   
2020-10-24      3.882574e-10  0.006401  1.162800e-02  0.000038  ...  0.015625   
2020-10-25      1.941287e-10  0.003201  5.813999e-03  0.000019  ...  0.007813   

Province_State                                                         \
Admin2         Platte      Sheridan Sublette Sweetwater Teton   Uinta   
2020-10-23        0.0  1.091394e-11      0.0        0.0   0.0  0.2500   
2020-10-24        0.0  5.456968e-12      0.0        0.0   0.0  0.1250   
2020-10-25        0.0  2.728484e-12      0.0        0.0   0.0  0.0625   

Province_State                              
Admin2         Unassigned  Washakie Weston  
2020-10-23      -0.999573  0.000081    NaN  
2020-10-24      -0.999573  0.000041    NaN  
2020-10-25      -0.999573  0.000020    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201025,AK,14064,NaN,545905,NaN,totalTestsViral,559969,58.0,NaN,...,559969,0,0,175a4f70a15c098ac277246bff6ae33838f66029,0,0,0,0,0,NaN
1,20201025,AL,184355,26474.0,1144785,NaN,totalTestsViral,1302666,922.0,19595.0,...,1329140,0,0,14421edcbba78df1217731644f92a3a03211c44d,0,0,0,0,0,NaN
2,20201025,AR,106115,7235.0,1189120,NaN,totalTestsViral,1288000,618.0,6718.0,...,1295235,15,11,813725c51fabd6f6078ad953d35346c5d5987dc1,0,0,0,0,0,NaN
3,20201025,AS,0,NaN,1616,NaN,totalTestsViral,1616,NaN,NaN,...,1616,0,0,3c89978d48f3a4f830660f7149129437b5808f1e,0,0,0,0,0,NaN
4,20201025,AZ,238163,5462.0,1473706,NaN,totalTestsPeopleViral,1706407,828.0,21086.0,...,1711869,5,43,7b19d3005937c11aaefe84f6c137e0ae59da6655,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201025,AK,14064.0,NaN,545905.0,NaN,totalTestsViral,559969.0,58.0,NaN,...,559969,0,0,175a4f70a15c098ac277246bff6ae33838f66029,0,0,0,0,0,NaN
1,20201025,AL,184355.0,26474.0,1144785.0,NaN,totalTestsViral,1302666.0,922.0,19595.0,...,1329140,0,0,14421edcbba78df1217731644f92a3a03211c44d,0,0,0,0,0,NaN
2,20201025,AR,106115.0,7235.0,1189120.0,NaN,totalTestsViral,1288000.0,618.0,6718.0,...,1295235,15,11,813725c51fabd6f6078ad953d35346c5d5987dc1,0,0,0,0,0,NaN
3,20201025,AS,0.0,NaN,1616.0,NaN,totalTestsViral,1616.0,NaN,NaN,...,1616,0,0,3c89978d48f3a4f830660f7149129437b5808f1e,0,0,0,0,0,NaN
4,20201025,AZ,238163.0,5462.0,1473706.0,NaN,totalTestsPeopleViral,1706407.0,828.0,21086.0,...,1711869,5,43,7b19d3005937c11aaefe84f6c137e0ae59da6655,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-10-25,AK,14064,NaN,545905,NaN,totalTestsViral,559969,58.0,NaN,NaN,...,559969,0,0,175a4f70a15c098ac277246bff6ae33838f66029,0,0,0,0,0,NaN
2020-10-25,AL,184355,26474.0,1144785,NaN,totalTestsViral,1302666,922.0,19595.0,NaN,...,1329140,0,0,14421edcbba78df1217731644f92a3a03211c44d,0,0,0,0,0,NaN
2020-10-25,AR,106115,7235.0,1189120,NaN,totalTestsViral,1288000,618.0,6718.0,244.0,...,1295235,15,11,813725c51fabd6f6078ad953d35346c5d5987dc1,0,0,0,0,0,NaN
2020-10-25,AS,0,NaN,1616,NaN,totalTestsViral,1616,NaN,NaN,NaN,...,1616,0,0,3c89978d48f3a4f830660f7149129437b5808f1e,0,0,0,0,0,NaN
2020-10-25,AZ,238163,5462.0,1473706,NaN,totalTestsPeopleViral,1706407,828.0,21086.0,181.0,...,1711869,5,43,7b19d3005937c11aaefe84f6c137e0ae59da6655,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-10-25,AK,14064.0,NaN,545905.0,NaN,totalTestsViral,559969.0,58.0,NaN,NaN,...,559969,0,0,175a4f70a15c098ac277246bff6ae33838f66029,0,0,0,0,0,NaN
2020-10-25,AL,184355.0,26474.0,1144785.0,NaN,totalTestsViral,1302666.0,922.0,19595.0,NaN,...,1329140,0,0,14421edcbba78df1217731644f92a3a03211c44d,0,0,0,0,0,NaN
2020-10-25,AR,106115.0,7235.0,1189120.0,NaN,totalTestsViral,1288000.0,618.0,6718.0,244.0,...,1295235,15,11,813725c51fabd6f6078ad953d35346c5d5987dc1,0,0,0,0,0,NaN
2020-10-25,AS,0.0,NaN,1616.0,NaN,totalTestsViral,1616.0,NaN,NaN,NaN,...,1616,0,0,3c89978d48f3a4f830660f7149129437b5808f1e,0,0,0,0,0,NaN
2020-10-25,AZ,238163.0,5462.0,1473706.0,NaN,totalTestsPeopleViral,1706407.0,828.0,21086.0,181.0,...,1711869,5,43,7b19d3005937c11aaefe84f6c137e0ae59da6655,0,0,0,0,0,NaN
